## Equipo 2
10/03/2023
### Proyecto Horario
#### OPTIMIZACIÓN DETERMINISTA
Codigo para optimizar el costo total dado por cada horario. El cual toma en cuenta diferente salones, horarios , materias y profesores.

Hecho por :
* Alan Uriel Merlan Esquivel A01656612
* Takeshi Nakauma Aguirre A01656026
* Dabria Camila Carillo Meneses A01656716
* Iker Sebastian Bali Elizalde A01656437
* Victor Augusto Godinez Velasco A01656075

In [1]:
# Programa de optimización de horarios
# Creamos las fuciones que vamos a utilizar para generar las restricciones extraidas de la API
def qn(list): # funcion para quitar los elementos NONE que puedan aparecer en nuestra API
 list2 = []
 for val in list:
  if val != None:
   list2.append(val)
 return list2

def aresm(dictp,listp): # funcion para agregar las restricciones de las materias
 for val in listp:
  for i in range(9):
   for j in range(10):
    if val == c[i+1, j].value:
     if c[i+1,j+4].value == None:
      dictp[val] = list((c.range('E14').value))
     else:
      dictp[val] = list((c[i+1,j+4].value))
   del j
  del i
 del val
 return dictp

def aresh(dicth,listp): # funcion para agregar las restricciones de los horarios
 for val in listp:
  for i in range(9):
   for j in range(10):
    if val == c[i+1,j].value:
     if c[i+1,j+5].value == None:
      dicth[val] = list(str(c.range('F14').value))
     else:
      dicth[val] = list(str(int(c[i+1,j+5].value)))
   del j
  del i
 del val
 return dicth

def acos(dict,lista,t): # funcion para agregar los costos
 for val in lista:
  if t == 's':
   for i in range(len(lista)+1):
    for j in range(10):
     if val == c[i+1,j].value:
      lista_temp = []
      if c[i+1,j+6].value == None:
       lista_temp.append((c.range('G14').value))
      else:
       lista_temp.append((c[i+1,j+6].value))
      dict[val] = list(map(int,lista_temp))
    del j
   del i
  else:
   for i in range(10):
    for j in range(len(lista)+1):
     if val == c[i+1,j].value:
      lista_temp = []
      if c[i+1,j+6].value == None:
       lista_temp.append((c.range('G14').value))
      else:
       lista_temp.append((c[i+1,j+6].value))
      dict[val] = list(map(int,lista_temp))
    del j
   del i
 del val
 return dict

#Importamos las librerias
import pulp
import xlwings as xl

# creamos nuestra función que hará el problema de optimización
def asignaciones(t_l,m_l,h_l,s_l,r_profe_materia,r_profe_horario,costo_profe,costo_salon,costo_horario,costo_materia):
  
 # Iniciamos el problema en pulp de tal forma que minimice nuestra función objetivo
 prob = pulp.LpProblem("Teachers", sense = pulp.const.LpMinimize)

 #Usamos pulp para crear todas las variables con las combinaciónes posibles dadas nuestros horarios.
 variables = pulp.LpVariable.dicts("X_",(t_l,m_l,h_l,s_l),0, None, pulp.const.LpBinary)
  
 #FUNCIÓN OBJETIVO.
 #Fo es una variable que sirve para acumular los diferentes costos datos por los profesores, materias, horarios y salones
 fo = pulp.lpSum([])
  
 #Esta parte agrega el costo por profesor. El proceso es que
 # va por cada profesor y checa cada costo para luego multiplicarlo por la variable correspondiente.
 if costo_profe != None:
  for t in t_l:
   for i in costo_profe[t]:  
    aux =  pulp.lpSum([i*variables[t][m][h][s] for m in m_l for h in h_l for s in s_l])
   fo = fo+aux
   aux  = pulp.lpSum([])
  del i

 #Luego, agregamos el costo por salon(el proceso es el mismo)
 if costo_salon != None:
  for s in s_l:
   for i in costo_salon[s]:
    aux =  pulp.lpSum([i*variables[t][m][h][s] for m in m_l for h in h_l for t in t_l])
   fo = fo+aux
   aux = pulp.lpSum([])
  del i
  
 #Agregamos el costo por horario
 if costo_horario != None:
  for h in h_l:
   for i in costo_horario[h]:  
    aux =  pulp.lpSum([i*variables[t][m][h][s] for m in m_l for t in t_l for s in s_l])
   fo = fo+aux
   aux  = pulp.lpSum([])
  del i

  #Agregamos el costo por materia
 if costo_materia != None:
  for m in m_l:
   for i in costo_materia[m]:  
    aux =  pulp.lpSum([i*variables[t][m][h][s] for h in h_l for t in t_l for s in s_l])
   fo = fo+aux
   aux  = pulp.lpSum([])
  del i

 #Y el resultado es una función que toma en cuenta todos los costos
 prob+= fo

 #En caso de que ningun costo es dado, se asegura de que aun funcione
 if (costo_profe == None) and (costo_materia == None) and (costo_horario == None) and (costo_salon == None):
  prob += pulp.lpSum([variables[t][m][h][s] for h in h_l for t in t_l for s in s_l for m in m_l]) 

 #Restricción 1 _ Una materia {m} solo puede tener un salón, un horario y un profesor
 #Para lograr lo anterior, nos aseguramos por medio del primer for que cada m(de manera que la m por restricción es constante)
 #  se separe.
 # Por lo que cada materia m solo puede tener un salón, horario y profesor.
 #El proceso es el mismo para el resto de restricciones. Por tanto, solo se mencionará que variable
 # se mantiene constante en cada restricción cuando se explique el funcionamiento.
 for m in m_l: 
  prob += (pulp.lpSum([variables[t][m][h][s] for t in t_l 
                      for h in h_l
                      for s in s_l]) == 1,
                      f" Restricción 1: Una materia {m} solo puede tener un salón, un horario y un profesor ")

 #Restricción 2. "Restricción 2: El profesor {t} está capacitado para dar la materia {m} 
 # Para esto, se mantiene la t constante. Y por cada diferente t(profesor) vericamos 
 #que materia pueda dar observando el diccionario r_profe_materia atraves del segundo for.
 for t in t_l:
  for m in r_profe_materia[t]:
   prob += (pulp.lpSum([variables[t][m][h][s] for h in h_l for s in s_l])) <= 1, f"Restricción 2: Un profesor {t} está capacitado para la materia {m} "

 #Restricción 3: El profesor {t} NO puede dar {m}"
 # Aquí igual mantenemos t constante. Ahora pasamos por tadas las materias y verificamos que el profesor no las de.
 # Si es así, agregamos una restricción para que ese profesor no de esa materia
 for t in t_l:
  for m in m_l:
   if (m in r_profe_materia[t]) == 0:
    prob += (pulp.lpSum([variables[t][m][h][s] 
                        for h in h_l 
                        for s in s_l]) == 0,
                        f"Restricción 3: El profesor {t} NO puede dar {m}")

 #Restricción4. Un profesor {t} está disponible en el horario {h}
 # Para esto, se mantiene la t constante. Y por cada diferente t(profesor) vericamos 
 #que horario puede dar observando el diccionario r_profe_horario atraves del segundo for.
 for t in t_l:
  for h in r_profe_horario[t]:
   prob += pulp.lpSum([variables[t][m][h][s] for m in m_l for s in s_l]) <= 1, f"Restricción 4: Un profesor {t} está disponible en el horario {h}"

 # Restricción 5: El profesor {t} no puede dar clase en el horario {h}
 # Mantenemos t constante. Ahora pasamos por tadas los horarios y verificamos que el profesor no este disponible.
 # Si es así, agregamos una restricción que se asegure que nunca de en ese horario a traves del segundo for

 for t in t_l: 
  for h in h_l:
   if (h in r_profe_horario[t]) == 0:
    prob += (pulp.lpSum([variables[t][m][h][s] for m in m_l for s in s_l])) == 0, f"Restricción 5: El profesor {t} no puede dar clase en el horario {h}"

 #Restriccion 6 Restricción 6: El salón {s} solo puede ser ocupado una vez en el horario {h}"
 #s,h constante
 for s in s_l:
  for h in h_l:
   prob += (pulp.lpSum([variables[t][m][h][s] for m in m_l for t in t_l]) <= 1, f"Restricción 6: El salón {s} solo puede ser ocupado una vez en el horario {h}")
  
 #Restriccion 7. Restricción 7: Un profesor {t} solo pueda dar una materia en horario {h}
 # t,h constante
 for t in t_l:
  for h in h_l:
   prob +=  pulp.lpSum([variables[t][m][h][s] for m in m_l for s in s_l  ]) <= 1, f"Restricción 7: Un profesor {t} solo pueda dar una materia en horario {h}"

 #En base a las restricciones y la función objetivo, resuelve el problema
 status = prob.solve()
 #Adjunta los combinaciones que dan el costo minimo en una lista.
 l_resultado = []
 for t in t_l:
  for m in m_l:
   for h in h_l:
    for s in s_l: 
     if variables[t][m][h][s].value() == 1 :
      l_resultado.append([t,m,h,s])

 print(f"Entonces el minimo costo posible es: {pulp.value(prob.objective)}")
  
 #Regresa una lista con los datos de que minimizan los costos.
 return l_resultado, (pulp.value(prob.objective))

# Mandamos a llamar el excel donde ponemos los datos de los profesores, materias, horarios y salones con sus respectivas restricciones
wb = xl.Book(r'Datos_profesores.xlsx')
c=wb.sheets[0] # cuaderno donde se encuentran nuestros datos

# Datos recolectados desde excel
c.range('B17:E30').value = '' # reiniciamos los resultados 
c.range('F19').value = '' 

# Listas de profesores, materias, horarios y salones
prof = qn(list((c.range('A2:A10').value)))
mat  = qn(list((c.range('B2:B10').value)))
hor1 = list(map(str,list(map(int,qn(list((c.range('C2:C10').value)))))))
hor  = qn(list((c.range('C2:C10').value)))
sal  = qn(list((c.range('D2:D10').value)))

# Restricciones de las materias que puede dar cada profesor
rpm = dict()
rpm = aresm(rpm,prof)

# Restricciones de los horarios que puede impartir cada profesor
rph = dict()
rph = aresh(rph,prof)

# Costos de profesores, materias, horarios y salones
cp = dict()
cp = acos(cp,prof,'p')
cm = dict()
cm = acos(cm,mat, 'm')
ch = dict()
ch = acos(ch,hor, 'h')
ch_temp = list(map(str,list(map(int,ch))))
for val in ch_temp:
 ch[val] = ch.pop(float(val))
del ch_temp
cs = dict()
cs = acos(cs,sal,'s')

resultado, costo_fin = asignaciones(prof,mat,hor1,sal,rpm,rph,cp,cs,ch,cm)
print(resultado)

# Vamos a pegar nuestro resultado al Excel
c.range('B17').value = resultado
c.range('F19').value = costo_fin

del c,ch,cm,cp,cs,hor,hor1,mat,prof,sal,resultado,rph,rpm,val,wb,costo_fin

Entonces el minimo costo posible es: 410.0
[['A', 'h', '1', 'b'], ['A', 'j', '4', 'b'], ['A', 'l', '3', 'b'], ['B', 'm', '6', 'b'], ['C', 'n', '8', 'b'], ['F', 'o', '6', 'e'], ['F', 'b', '7', 'b'], ['G', 'r', '9', 'b'], ['G', 'e', '2', 'b']]
